#### Analysis of Load and Generator MW and MVAR

The following analysis is carried out for the obtained PSSE PV analysis results for the 2 studied cases

1. Without the Shunt at Bus 205  - (with the PV excel file - book_wo.xlsx)
2. With the Shunt of 300 MVAR connected at Bus 205 - (with the PV excel file - book_shunt.xlsx)


In [1]:
import pandas as pd
import math

In this analysis, corresponding to each power factors maximum incremental transfer and any desired incremental transfer data is extracted for

- Real Power Consumption of Loads
- Reactive Power Consumption of Loads
- Real Power Generation at Bus 211 HYDRO_G
- Reactive Power Generation at Bus 211 HYDRO_G
- Reactive Power Generation at Bus 206 URBGEN
- Real Power Generation at Bus 206 URBGEN


Function to extract Connected load MW and MVAR at maximum possible incremental transfer 

In [2]:
def function_load(book_name):
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    load_sheets =  []
    df_load_list = []
    for load in loads:
        sheet_name = str(load) + ' Bus Load'
        load_sheets.append(sheet_name)
    for load_sheet in load_sheets:
        data = pd.read_excel(book_name ,sheet_name = load_sheet, header=2)
        data[['Unnamed: 0', 'Unnamed: 1']] = data[['Unnamed: 0', 'Unnamed: 1']].astype(str)
        data_bus = data.iloc[:,:2]
        data['Load Bus Name'] = data_bus[['Unnamed: 0', 'Unnamed: 1']].agg(' '.join, axis=1)
        data_power = data.iloc[:,-3:]
        data_power = data_power.rename(columns={data_power.columns[0]: "Load MW", data_power.columns[1]: "Load MVAR"})
        data_power['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(load_sheet.split(' ')[0])/20))),2))
        df_load_list.append(data_power)
    df_load = pd.concat(df_load_list).reset_index(drop=True)
    return df_load

Function to extract Connected load MW and MVAR for desired incremental transfer 

In [3]:
def function_load_desired(book_name, desired_power):
    import numpy as np
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    load_sheets =  []
    df_load_list = []
    for load in loads:
        sheet_name = str(load) + ' Bus Load'
        load_sheets.append(sheet_name)
    for load_sheet in load_sheets:
        data = pd.read_excel(book_name ,sheet_name = load_sheet, header=2)
        data[['Unnamed: 0', 'Unnamed: 1']] = data[['Unnamed: 0', 'Unnamed: 1']].astype(str)
        data_bus = data.iloc[:,:2]
        data['Load Bus Name'] = data_bus[['Unnamed: 0', 'Unnamed: 1']].agg(' '.join, axis=1)
        data_power = data
        n = str(int(desired_power/20)-1)
        mw_name = 'MW.' + n
        mvar_name ='MVAR.' +n
        if mw_name in data_power:
            data_power = data[['Load Bus Name',mw_name, mvar_name]]
        else:
            data_power[mw_name] = np.nan
            data_power[mvar_name] = np.nan
            data_power = data[['Load Bus Name',mw_name, mvar_name]]
        data_power = data_power.rename(columns={data_power.columns[1]: "Load MW", data_power.columns[2]: "Load MVAR"})
        data_power['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(load_sheet.split(' ')[0])/20))),2))
        df_load_list.append(data_power)
    df_load = pd.concat(df_load_list).reset_index(drop=True)
    return df_load

Function to extract Generator MW and MVAR at maximum possible incremental transfer from the pv analysis excel file result

In [4]:
def function_gen(book_name):
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    gen_sheets = []
    for gen in loads:
        sheet = str(gen) + ' Generator Dispatch'
        gen_sheets.append(sheet)  
    gen_df_list = []
    for gen_sheet in gen_sheets:
        data = pd.read_excel(book_name,sheet_name = gen_sheet, header=2)
        data[['Unnamed: 0', 'Unnamed: 1']] = data[['Unnamed: 0', 'Unnamed: 1']].astype(str)
        data_bus = data.iloc[:,:2]
        data['Generator Bus Name'] = data_bus[['Unnamed: 0', 'Unnamed: 1']].agg(' '.join, axis=1)
        data_power = data.iloc[:,-3:]
        data_power = data_power.rename(columns={data_power.columns[0]: "Generator MW", data_power.columns[1]: "Generator MVAR"})
        data_power['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(gen_sheet.split(' ')[0])/20))),2))
        gen_df_list.append(data_power)
    df_gen = pd.concat(gen_df_list).reset_index(drop=True)
    return df_gen

Function to extract generator MW and MVAR for desired incremental transfer 

In [5]:
def function_gen_desired(book_name, desired_power):
    import numpy as np
    loads = [39.69,34.64,30.36,26.66,23.38,20.40,17.63,15,12.39,10.79,9.68,0]
    load_sheets =  []
    df_gen_list = []
    for load in loads:
        sheet_name = str(load) + ' Generator Dispatch'
        load_sheets.append(sheet_name)
    for load_sheet in load_sheets:
        data = pd.read_excel(book_name ,sheet_name = load_sheet, header=2)
        data[['Unnamed: 0', 'Unnamed: 1']] = data[['Unnamed: 0', 'Unnamed: 1']].astype(str)
        data_bus = data.iloc[:,:2]
        data['Generator Bus Name'] = data_bus[['Unnamed: 0', 'Unnamed: 1']].agg(' '.join, axis=1)
        data_power = data
        n = str(int(desired_power/20))
        mw_name = 'MW.' + n
        mvar_name ='MVAR.' +n
        if mw_name in data_power:
            data_power = data[['Generator Bus Name',mw_name, mvar_name]]
        else:
            data_power[mw_name] = np.nan
            data_power[mvar_name] = np.nan
            data_power = data[['Generator Bus Name',mw_name, mvar_name]]
        data_power = data_power.rename(columns={data_power.columns[1]: "Generator MW", data_power.columns[2]: "Generator MVAR"})
        data_power['Load PF'] = 'PF = ' + str(round(math.cos((math.atan(float(load_sheet.split(' ')[0])/20))),2))
        df_gen_list.append(data_power)
    df_gen = pd.concat(df_gen_list).reset_index(drop=True)
    return df_gen

#### Case 1. Without the Shunt at Bus 205 - (with the PV excel file - book_wo.xlsx)

1. Load MW and Load MVAR value for maximum incremental transfer possible 

In [6]:
data_load_MIT = function_load('book_wo.xlsx')

In [7]:
list_load = list(data_load_MIT['Load Bus Name'].unique())

In [8]:
for load in list_load:
    display(data_load_MIT[data_load_MIT['Load Bus Name'] == load])

,Load MW,Load MVAR,Load Bus Name,Load PF
0,300.0,150.0,203 EAST230,PF = 0.45
2,300.0,150.0,203 EAST230,PF = 0.5
4,300.0,150.0,203 EAST230,PF = 0.55
6,300.0,150.0,203 EAST230,PF = 0.6
8,300.0,150.0,203 EAST230,PF = 0.65
10,300.0,150.0,203 EAST230,PF = 0.7
12,300.0,150.0,203 EAST230,PF = 0.75
14,300.0,150.0,203 EAST230,PF = 0.8
16,300.0,150.0,203 EAST230,PF = 0.85
18,300.0,150.0,203 EAST230,PF = 0.88


,Load MW,Load MVAR,Load Bus Name,Load PF
1,420.000031,833.489990,205 SUB230,PF = 0.45
3,479.999969,831.359985,205 SUB230,PF = 0.5
5,539.999939,819.719910,205 SUB230,PF = 0.55
7,619.999817,826.459778,205 SUB230,PF = 0.6
9,699.999756,818.299683,205 SUB230,PF = 0.65
11,789.999695,805.799683,205 SUB230,PF = 0.7
13,889.999695,784.534668,205 SUB230,PF = 0.75
15,1019.999573,764.999695,205 SUB230,PF = 0.8
17,1169.999390,724.814575,205 SUB230,PF = 0.85
19,1279.999268,690.559570,205 SUB230,PF = 0.88


As PV analysis is done by considering the sink as the connected load at Bus 205 SUB230, the results above show that for connected load at Bus 203, there is no incremental transfer and hence is constant for all studied power factors. For the connected load at Bus 205, for the studied powerfactors the load variation is as obtained and the maximum MW and MVAR that can be transfered to the load are observed. As can be seen for PF = 0.45, a maximum incremental transfer of 420 MW is possible to the connected load at Bus 205 and as it is a highly reactive load there is it has an MVAR requirement of 833.5 MVAR. 

1. Generator Dispatch MW and MVAR value for maximum incremental transfer possible 

In [9]:
data_gen_MIT = function_gen('book_wo.xlsx')

In [10]:
list_gen = list(data_gen_MIT['Generator Bus Name'].unique())

In [11]:
for gen in list_gen:
    display(data_gen_MIT[data_gen_MIT['Generator Bus Name'] == gen])

,Generator MW,Generator MVAR,Generator Bus Name,Load PF
0,39.852154,600.000000,206 URBGEN,PF = 0.45
2,99.926689,600.000000,206 URBGEN,PF = 0.5
4,159.147064,600.000000,206 URBGEN,PF = 0.55
6,239.765854,600.000000,206 URBGEN,PF = 0.6
8,319.105164,600.000000,206 URBGEN,PF = 0.65
10,408.924927,600.000000,206 URBGEN,PF = 0.7
12,508.953186,600.000000,206 URBGEN,PF = 0.75
14,639.053650,600.000000,206 URBGEN,PF = 0.8
16,789.071777,600.000000,206 URBGEN,PF = 0.85
18,899.077393,600.000000,206 URBGEN,PF = 0.88


,Generator MW,Generator MVAR,Generator Bus Name,Load PF
1,616.25,400.000000,211 HYDRO_G,PF = 0.45
3,616.25,400.000000,211 HYDRO_G,PF = 0.5
5,616.25,400.000000,211 HYDRO_G,PF = 0.55
7,616.25,400.000000,211 HYDRO_G,PF = 0.6
9,616.25,400.000000,211 HYDRO_G,PF = 0.65
11,616.25,400.000000,211 HYDRO_G,PF = 0.7
13,616.25,400.000000,211 HYDRO_G,PF = 0.75
15,616.25,400.000000,211 HYDRO_G,PF = 0.8
17,616.25,400.000000,211 HYDRO_G,PF = 0.85
19,616.25,400.000000,211 HYDRO_G,PF = 0.88


The Generation in area 2 is serving the connected loads at bus 203, 205, +/- inter area transfers to area 2, and the losses. As economic dispatch is considered, the generator at bus 211 is committed first and then only the generator at Bus 206 committed for generation based on the load requirement. For lower power factors, the generation requirement is completely served by Generator at Bus 211 and for higher power factors the generation contribution from unit at bus 206 is more to serve the higher load demand.

2. Desired incremental transfer of 600 MW

In [12]:
data_load_600 = function_load_desired('book_wo.xlsx',600)

In [13]:
for load in list_load:
    display(data_load_600[data_load_600['Load Bus Name'] == load])

,Load Bus Name,Load MW,Load MVAR,Load PF
0,203 EAST230,NaN,NaN,PF = 0.45
2,203 EAST230,NaN,NaN,PF = 0.5
4,203 EAST230,NaN,NaN,PF = 0.55
6,203 EAST230,300.0,150.0,PF = 0.6
8,203 EAST230,300.0,150.0,PF = 0.65
10,203 EAST230,300.0,150.0,PF = 0.7
12,203 EAST230,300.0,150.0,PF = 0.75
14,203 EAST230,300.0,150.0,PF = 0.8
16,203 EAST230,300.0,150.0,PF = 0.85
18,203 EAST230,300.0,150.0,PF = 0.88


,Load Bus Name,Load MW,Load MVAR,Load PF
1,205 SUB230,NaN,NaN,PF = 0.45
3,205 SUB230,NaN,NaN,PF = 0.5
5,205 SUB230,NaN,NaN,PF = 0.55
7,205 SUB230,599.999878,799.799866,PF = 0.6
9,205 SUB230,599.999878,701.399780,PF = 0.65
11,205 SUB230,599.999878,611.999817,PF = 0.7
13,205 SUB230,599.999878,528.899841,PF = 0.75
15,205 SUB230,599.999878,449.999908,PF = 0.8
17,205 SUB230,599.999878,371.699890,PF = 0.85
19,205 SUB230,599.999878,323.699921,PF = 0.88


As already observed for 
1. PF = 0.45 - maximum incremental transfer possible is 400 MW with Load MW at bus 205 of 420 MW
2. PF = 0.5 - maximum incremental transfer possible is 460 MW with Load MW at bus 205 of 480 MW
3. PF = 0.55 - maximum incremental transfer possible is 520 MW with Load MW at bus 205 of 540 MW

Hence an incremental transfer of 600 MW is not possible for these power factors. 
For the rest of the studied Power factors, for the same incremental transfer of 600 MW, the Load MVAR requirement decreases as Power factor increases

In [14]:
data_gen_600 = function_gen_desired('book_wo.xlsx',600)

In [15]:
for gen in list_gen:
    display(data_gen_600[data_gen_600['Generator Bus Name'] == gen])

,Generator Bus Name,Generator MW,Generator MVAR,Load PF
0,206 URBGEN,NaN,NaN,PF = 0.45
2,206 URBGEN,NaN,NaN,PF = 0.5
4,206 URBGEN,NaN,NaN,PF = 0.55
6,206 URBGEN,229.765869,600.000000,PF = 0.6
8,206 URBGEN,239.105148,600.000000,PF = 0.65
10,206 URBGEN,238.924881,600.000000,PF = 0.7
12,206 URBGEN,228.797531,600.000000,PF = 0.75
14,206 URBGEN,228.791504,600.000000,PF = 0.8
16,206 URBGEN,228.785614,595.075012,PF = 0.85
18,206 URBGEN,228.781769,538.258484,PF = 0.88


,Generator Bus Name,Generator MW,Generator MVAR,Load PF
1,211 HYDRO_G,NaN,NaN,PF = 0.45
3,211 HYDRO_G,NaN,NaN,PF = 0.5
5,211 HYDRO_G,NaN,NaN,PF = 0.55
7,211 HYDRO_G,616.25,400.000000,PF = 0.6
9,211 HYDRO_G,616.25,285.664612,PF = 0.65
11,211 HYDRO_G,616.25,158.986298,PF = 0.7
13,211 HYDRO_G,616.25,56.612061,PF = 0.75
15,211 HYDRO_G,616.25,-33.751808,PF = 0.8
17,211 HYDRO_G,616.25,-100.000000,PF = 0.85
19,211 HYDRO_G,616.25,-100.000000,PF = 0.88


The load requirements are served with generation from units at Bus 211 and 206, and interchange from Area 1

#### Case 2. With the Shunt at Bus 205 - (with the PV excel file - book_shunt.xlsx)

1. Load MW and Load MVAR value for maximum incremental transfer possible 

In [16]:
data_load_MIT_shunt = function_load('book_shunt.xlsx')

In [17]:
list_load = list(data_load_MIT_shunt['Load Bus Name'].unique())

In [18]:
for load in list_load:
    display(data_load_MIT_shunt[data_load_MIT_shunt['Load Bus Name'] == load])

,Load MW,Load MVAR,Load Bus Name,Load PF
0,300.0,150.0,203 EAST230,PF = 0.45
2,300.0,150.0,203 EAST230,PF = 0.5
4,300.0,150.0,203 EAST230,PF = 0.55
6,300.0,150.0,203 EAST230,PF = 0.6
8,300.0,150.0,203 EAST230,PF = 0.65
10,300.0,150.0,203 EAST230,PF = 0.7
12,300.0,150.0,203 EAST230,PF = 0.75
14,300.0,150.0,203 EAST230,PF = 0.8
16,300.0,150.0,203 EAST230,PF = 0.85
18,300.0,150.0,203 EAST230,PF = 0.88


,Load MW,Load MVAR,Load Bus Name,Load PF
1,519.999939,1031.939819,205 SUB230,PF = 0.45
3,589.999878,1021.879761,205 SUB230,PF = 0.5
5,669.999817,1017.059692,205 SUB230,PF = 0.55
7,759.999695,1013.079651,205 SUB230,PF = 0.6
9,849.999695,993.649597,205 SUB230,PF = 0.65
11,959.999573,979.199524,205 SUB230,PF = 0.7
13,1089.999512,960.834473,205 SUB230,PF = 0.75
15,1229.999390,922.499451,205 SUB230,PF = 0.8
17,1279.999268,792.959473,205 SUB230,PF = 0.85
19,1279.999268,690.559570,205 SUB230,PF = 0.88


With the addition of shunt at Bus 205, maximum possible incremental transfer is more for lower power factors compared to the results without shunt because of the additional MVAR provided by the shunt. Corresponding generation is as given

In [19]:
data_gen_MIT_shunt = function_gen('book_shunt.xlsx')
list_gen = list(data_gen_MIT_shunt['Generator Bus Name'].unique())
for gen in list_gen:
    display(data_gen_MIT_shunt[data_gen_MIT_shunt['Generator Bus Name'] == gen])

,Generator MW,Generator MVAR,Generator Bus Name,Load PF
0,136.997833,600.0,206 URBGEN,PF = 0.45
2,205.505295,600.0,206 URBGEN,PF = 0.5
4,284.332764,600.0,206 URBGEN,PF = 0.55
6,378.949005,600.0,206 URBGEN,PF = 0.6
8,468.638000,600.0,206 URBGEN,PF = 0.65
10,576.271545,600.0,206 URBGEN,PF = 0.7
12,705.286621,600.0,206 URBGEN,PF = 0.75
14,854.946594,600.0,206 URBGEN,PF = 0.8
16,893.415955,600.0,206 URBGEN,PF = 0.85
18,892.846313,600.0,206 URBGEN,PF = 0.88


,Generator MW,Generator MVAR,Generator Bus Name,Load PF
1,616.25,396.068329,211 HYDRO_G,PF = 0.45
3,616.25,384.254395,211 HYDRO_G,PF = 0.5
5,616.25,386.559052,211 HYDRO_G,PF = 0.55
7,616.25,396.741852,211 HYDRO_G,PF = 0.6
9,616.25,381.905518,211 HYDRO_G,PF = 0.65
11,616.25,382.348206,211 HYDRO_G,PF = 0.7
13,616.25,395.253784,211 HYDRO_G,PF = 0.75
15,616.25,385.542603,211 HYDRO_G,PF = 0.8
17,616.25,171.576935,211 HYDRO_G,PF = 0.85
19,616.25,20.589348,211 HYDRO_G,PF = 0.88


In [20]:
data_load_600_shunt = function_load_desired('book_shunt.xlsx',600)

In [21]:
for load in list_load:
    display(data_load_600_shunt[data_load_600_shunt['Load Bus Name'] == load])

,Load Bus Name,Load MW,Load MVAR,Load PF
0,203 EAST230,NaN,NaN,PF = 0.45
2,203 EAST230,300.0,150.0,PF = 0.5
4,203 EAST230,300.0,150.0,PF = 0.55
6,203 EAST230,300.0,150.0,PF = 0.6
8,203 EAST230,300.0,150.0,PF = 0.65
10,203 EAST230,300.0,150.0,PF = 0.7
12,203 EAST230,300.0,150.0,PF = 0.75
14,203 EAST230,300.0,150.0,PF = 0.8
16,203 EAST230,300.0,150.0,PF = 0.85
18,203 EAST230,300.0,150.0,PF = 0.88


,Load Bus Name,Load MW,Load MVAR,Load PF
1,205 SUB230,NaN,NaN,PF = 0.45
3,205 SUB230,589.999878,1021.879761,PF = 0.5
5,205 SUB230,599.999878,910.799805,PF = 0.55
7,205 SUB230,599.999878,799.799866,PF = 0.6
9,205 SUB230,599.999878,701.399780,PF = 0.65
11,205 SUB230,599.999878,611.999817,PF = 0.7
13,205 SUB230,599.999878,528.899841,PF = 0.75
15,205 SUB230,599.999878,449.999908,PF = 0.8
17,205 SUB230,599.999878,371.699890,PF = 0.85
19,205 SUB230,599.999878,323.699921,PF = 0.88


With the addition of shunt at Bus 205, the desired incremental transfer of 600 MW is possible for PF = 0.55 and above, and for PF = 0.5, an incremental transfer of 590 MW is possible. Corresponding generation is as given 

In [22]:
data_gen_600_shunt = function_gen_desired('book_shunt.xlsx',600)
for gen in list_gen:
    display(data_gen_600_shunt[data_gen_600_shunt['Generator Bus Name'] == gen])

,Generator Bus Name,Generator MW,Generator MVAR,Load PF
0,206 URBGEN,NaN,NaN,PF = 0.45
2,206 URBGEN,NaN,NaN,PF = 0.5
4,206 URBGEN,234.332748,600.000000,PF = 0.55
6,206 URBGEN,228.138992,600.000000,PF = 0.6
8,206 URBGEN,237.342926,600.000000,PF = 0.65
10,206 URBGEN,236.271637,546.208374,PF = 0.7
12,206 URBGEN,235.286880,449.655212,PF = 0.75
14,206 URBGEN,234.342545,359.872070,PF = 0.8
16,206 URBGEN,233.416336,272.586456,PF = 0.85
18,206 URBGEN,232.846710,219.973633,PF = 0.88


,Generator Bus Name,Generator MW,Generator MVAR,Load PF
1,211 HYDRO_G,NaN,NaN,PF = 0.45
3,211 HYDRO_G,NaN,NaN,PF = 0.5
5,211 HYDRO_G,616.25,245.171097,PF = 0.55
7,211 HYDRO_G,616.25,76.918983,PF = 0.6
9,211 HYDRO_G,616.25,-57.550613,PF = 0.65
11,211 HYDRO_G,616.25,-100.000000,PF = 0.7
13,211 HYDRO_G,616.25,-100.000000,PF = 0.75
15,211 HYDRO_G,616.25,-100.000000,PF = 0.8
17,211 HYDRO_G,616.25,-100.000000,PF = 0.85
19,211 HYDRO_G,616.25,-100.000000,PF = 0.88


#### Conclusion 

The results are observed for MW and MVAR transfered to the connected loads and generated MW and MVAR from units connected at Bus 206 and 211 when shunt is connected at Bus 205 and when shunt is not connected at Bus 205. 